In [1]:
import requests

import creds

In [46]:
FIELDS = [
    'ad_creation_time',
    'ad_creative_body',
    'ad_creative_link_caption',
    'ad_creative_link_description',
    'ad_creative_link_title',
    'ad_delivery_start_time',
    'ad_delivery_stop_time',
    'ad_snapshot_url',
    'currency',
    'demographic_distribution',
    'funding_entity',
    'impressions',
    'page_id',
    'page_name',
    'region_distribution',
    'spend',
]


In [76]:
def make_request(after=None):
    params = {
        # 'ad-type': 'POLITICAL_AND_ISSUE_ADS' (default)
        'fields': ','.join(FIELDS),
        'search_terms': "''",
        'ad_reached_countries': "['GB']",
        'limit': 5000,
        'access_token': creds.FB_TOKEN,
    }
    if after:
        params['after'] = after

    response = requests.get(
        "https://graph.facebook.com/v3.3/ads_archive",
        params=params,
    )

    assert response.status_code == 200
    json_data = response.json()
    print('Got {} ads'.format(len(json_data['data'])))

    assert set(json_data) == {'data', 'paging'}, set(json_data)
    paging = json_data['paging']
    assert set(paging) == {'cursors', 'next'}, paging
    assert set(paging['cursors']) == {'after'}, paging

    return json_data['data'], json_data['paging']['cursors']['after']

In [80]:
ads, after = make_request()
print(after)

Got 2897 ads
bWFudWFsX2N1cnNvcjpNVFUwTURJeE9UVTNOem95T0RreE9EUTNNekU0TURVMk5EWT0ZD
